<a href="https://colab.research.google.com/github/Azuch/vintern_llm_ocr_colab_ngrok/blob/main/Vintern_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q transformers==4.44.2 bitsandbytes
!pip install -U -q huggingface_hub
!pip install -q flask flask-cors pyngrok flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [2]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import requests

Libs for input images

In [3]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(requests.get(image_file, stream=True).raw).convert('RGB')#Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

Load model va test model tren colab

In [5]:
model_name = "5CD-AI/Vintern-1B-v2"
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens= 512, do_sample=False, num_beams = 3, repetition_penalty=3.5)


model.safetensors:  69%|######9   | 1.30G/1.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [11]:
test_image = 'https://image.phunuonline.com.vn/fckeditor/upload/2024/20240623/images/chinh-thuc-co-mau-the-can-_211719111878.webp'

pixel_values = load_image(test_image, max_num=6).to(torch.bfloat16).cuda()

prompt = '''<image\> Nhận diện và trả vể các đề mục '''

response = model.chat(tokenizer, pixel_values, prompt, generation_config)

del pixel_values
response



Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


'Bức ảnh chụp cận cảnh một thẻ căn cước công dân Việt Nam.  Phần trên cùng của thẻ có dòng chữ "CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM" được in đậm, bên dưới là dòng chữ "Độc lập - Tự do - Hạnh phúc" và "SOCIALIST REPUBLIC OF VIET NAM (Independence - Freedom - Unity)". Tiếp theo là dòng chữ "CĂN CƯỚC CÔNG DÂN" được in màu đỏ nổi bật. Bên cạnh đó là hình ảnh quốc huy Việt Nam. Phía dưới là các thông tin cá nhân bao gồm: \n- Số / No.\n- Họ và tên / Full name:\n- Ngày sinh / Date of birth:\n- Giới tính / Sex: Nam\n- Quốc tịch / Nationality: Việt Nam\n- Quê quán / Place of origin:\n- Nơi thường trú / Place of residence'

In [12]:
# Setup Ngrok Token
from google.colab import userdata
from flask import Flask, jsonify, request
from flask_cors import CORS
from pyngrok import ngrok

authtoken = userdata.get("ngrok_token")
ngrok.set_auth_token(authtoken)


In [16]:
# Viết code Flask để expose ra API

app = Flask(__name__)
CORS(app)

prompt = "<image\> Trả các thông tin dưới dạng csv."

@app.route('/ocr', methods=['POST'])
def index():
    data = request.json
    image_url = data.get('image_url', None)

    response_message = ocr_by_llm(image_url, prompt)

    return jsonify({
        "response_message": response_message
    })

def ocr_by_llm(image_url, prompt):
    # image = Image.open(requests.get(image_url, stream=True).raw)

    pixel_values = load_image(image_url, max_num=6).to(torch.bfloat16).cuda()

    response_message = model.chat(tokenizer, pixel_values, prompt, generation_config)

    del pixel_values

    print(response_message)
    return response_message

if __name__ == "__main__":
    ngrok_url = ngrok.connect(5555)
    print(ngrok_url)

    app.run(port=5555)


NgrokTunnel: "https://f8cc-34-168-230-114.ngrok-free.app" -> "http://localhost:5555"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5555
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /ocr [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()


{
  "Cộng hòa xã hội chủ nghĩa Việt Nam": "Độc lập - Tự do - Hạnh phúc",
  "Citizen Identity Card": {
    "Số/No": "",
    "Họ và tên/Full name": "",
    "Ngày sinh/Date of birth": "",
    "Giới tính/Sex": "Nam",
    "Quốc tịch/Nationality": "Việt Nam",
    "Quê quán/Place of origin": "",
    "Nơi thường trú/Place of residence": ""
  }
}


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 09:18:42] "POST /ocr HTTP/1.1" 200 -


{
  "Tên": "Doãn Ngọc Hải",
  "Ngày sinh": "50 tuổi",
  "Chức vụ": "Phó Tổng Giám đốc Tổng Công ty Xây dựng Sài Gòn TNHH Một thành viên, theo Quyết định số 74QD-UBND-TC ngày 03 tháng 6 năm 2019 của Ủy ban nhân dân thành phố.",
  "Trình độ chuyên môn": "Thạc sĩ Chính trị, Cử nhân Luật, Cử nhân Kinh tế; Cử nhân Xã hội học.",
  "Ngày ký": "08 tháng 01 năm 2018",
  "Nội dung": "Tôi đã có đơn từ chức gửi các đồng chí Lãnh đạo thành phố và quận. Ngày 10 tháng 5 năm 2018, qua trao đổi tin nhắn với đồng chí Nguyễn Thiện Nhân - Ủy viên Bộ Chính trị, Bí thư Thành ủy đã động viên và thuyết phục tôi rút đơn từ chức. Tôi đã đồng ý và chấp hành rút đơn vào ngày 12 tháng 5 năm 2018."
}
